In [28]:
# requirements-list
# !pip install faster-whisper==1.0.0

In [1]:
!pip show torch

Name: torch
Version: 2.0.1+cu117
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /mnt/storage/kocharyan/sambashare/ITMO_LABS/ML/venv/lib/python3.8/site-packages
Requires: filelock, jinja2, networkx, sympy, triton, typing-extensions
Required-by: accelerate, adan-pytorch, asteroid-filterbanks, julius, lightning, openai-whisper, pyannote.audio, pytorch-lightning, pytorch-metric-learning, so-vits-svc-fork, speechbrain, thop, torch-audiomentations, torch-pitch-shift, torchaudio, torchcrepe, torchmetrics, torchvision, trainer, triton, ultralytics


In [2]:
!nvidia-smi

Fri Mar 15 11:28:45 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  Off |
|  0%   49C    P8    19W / 450W |      0MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from utils import del_folder, create_chunk_dataset
from hubert_proc import get_hubert, create_hubert_content
from clustering import cluster_training_on_data

from so_vits.modules.attentions import Encoder
import whisperx

from pathlib import Path
import os

import torch

os.environ["http_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["https_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["ftp_proxy"] = "http://proxy.ad.speechpro.com:3128"

/mnt/storage/kocharyan/sambashare/ITMO_LABS/ML/venv/local/lib/python3.8/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


ModuleNotFoundError: No module named 'whisperx'

In [6]:
def path(_path):
    return "./examples/" / Path(_path)

In [7]:
!ls ../NIR/RuDevices

0  1  2  3  4  5  6  7	8  9  a  b  c  d  e  f


In [16]:
create_chunk_dataset("../NIR/RuDevices", out_dataset=path("rudevices_chunk"), display=True)

rudevices_chunk/
├── 9/
│   ├── d/
│   │   ├── 22bc8786-a705-4c5e-a7c2-1062fd43961e.wav
│   │   ├── 682be63f-cce9-436d-9694-754d70e68499.wav
│   │   ├── 6f47e71e-55d8-4aa8-a059-eed74a5e6744.wav
│   │   └── bc9c83f8-964a-4a20-a930-3e3c98aeead8.wav
│   └── e/
│       ├── 685d1b73-8af8-4b19-b207-a52fe831b4a0.wav
│       ├── 8b0f97c8-55c8-4c97-9f68-9a091c1d8159.wav
│       ├── 906696b8-8c92-4225-8808-b08c9029c11a.wav
│       └── baa26e05-8782-4df8-8ade-262e50b3a34f.wav
└── d/
    ├── 3/
    │   ├── 671b7945-fd12-46d4-ae65-bbf2ad8caf90.wav
    │   ├── 946c0fe8-eb20-47e6-85d2-9ed8a5ed1795.wav
    │   ├── ab0f7026-2d61-4c71-a5ca-1f361457c740.wav
    │   └── eede5ec1-2865-4981-b20b-1b5b86a6bc83.wav
    └── 9/
        ├── 5b4aec41-5046-416b-8c32-4303fcceb49d.wav
        ├── 6bd71d75-bd7c-483f-b841-7ed934954088.wav
        ├── 84bb18fa-59e3-460c-946d-4af2426b44aa.wav
        └── c8b54f05-2a9e-40c1-87fd-7d67b6bd82f0.wav


In [13]:
# del_folder(Path("ali"))
create_hubert_content(data_dir=path("rudevices_chunk"), out_dir=path("ali")) # TODO: проблемы с whisperX

100%|█████████████████████████████████████| 2.87G/2.87G [02:37<00:00, 19.6MiB/s]
100%|█████████████████████████████████████| 2.87G/2.87G [02:43<00:00, 18.9MiB/s]
100%|█████████████████████████████████████| 2.87G/2.87G [02:57<00:00, 17.4MiB/s]


There is no default alignment model set for this language (ru).                Please find a wav2vec2.0 model finetuned on this language in https://huggingface.co/models, then pass the model name in --align_model [MODEL_NAME]


ValueError: No default align-model for language: ru

In [ ]:
cluster_training_on_data(path("ali"), path("clusters/clusters.pt"), 2, 2048, "*.content.pt")

In [ ]:



char_clusters = CharClusters("clusters/clusters.pt")
char_clusters.build_chars_clusters()

data = torch.load("clusters/clusters.pt")
_data = (data["а"]['cluster_centers'][0] + data["ф"]['cluster_centers'][0]).reshape(1, -1)
print(_data.shape)
c = char_clusters.get_cluster_center("а", _data)
print(c.shape)
_data, c


path = "ali/ё/rudevices_chunk.6.b.1baf3c9f-6672-4c32-abe6-e82f7ee9d40a.wav.content.pt"
torch.load(path)["content"].shape


file = 'rudevices_chunk/6/b/887fc29a-fdf8-4199-bfcd-2134c7fe1db8.wav'
f = torch.load("ali/align.pt")['ali'][file]
f


pprint([p for p in Path("rudevices_chunk/").rglob("*.wav")]) 

get_label_for_file('rudevices_chunk/1/4/9ef07055-3976-4d5c-96d3-6e41a52fc190.wav')


!cat rudevices_chunk/1/4/9ef07055-3976-4d5c-96d3-6e41a52fc190.txt


print(clac_label_res[0], clac_label_res[1], clac_label_res[1].count(')'), clac_label_res[2], sep='\n\n')


predict = hubert_inference('rudevices_chunk/6/b/887fc29a-fdf8-4199-bfcd-2134c7fe1db8.wav', hubert_inf, processor, "cuda")
predict.shape

display_tree("rudevices_chunk/")

!ls rudevices_chunk/1/4
processor, hubert_model, hubert_inf = get_hubert()

In [ ]:
files = [
    p.with_suffix(".txt").as_posix()[len("rudevices_chunk/"):] 
    for p in Path("rudevices_chunk/").rglob("*.wav")
]
# print(files)
for f in files:
    shutil.copy(Path("RuDevices") / f, Path("rudevices_chunk") / f)

In [ ]:
#TODO: create config

n_vocab         = 177 #52 # len(symbols) from text.symbols import symbols
inter_channels  = 192
hidden_channels = 192
filter_channels = 768
n_heads         = 2
n_layers        = 10 #6
kernel_size     = 3
p_dropout       = .1

prior_encoder = TextEncoder(
    n_vocab, inter_channels, hidden_channels, filter_channels, 
    n_heads, n_layers, kernel_size, p_dropout,
)

# prior_encoder()
# "vist_ckpt/G_0.pth"
# "enc_p"
prior_encoder = load_checkpoint(prior_encoder, 
                                "YourTTS-RU-RUSLAN-April-30-2023_03+48PM-0000000/best_model_158996.pth", 
                                "text_encoder", False)


In [ ]:
dataset = TextFromAudioDataset(
    "rudevices_chunk/", 
    "YourTTS-RU-RUSLAN-April-30-2023_03+48PM-0000000/config.json",
    processor, hubert_inf
)

loader = DataLoader(dataset, batch_size=1)
next(iter(loader))

In [ ]:
dataset.dataset[:5]

In [ ]:
# Почему len(symbols)==52 в исходной версии (по размеру эмбеддинга)
# return phonem
dataset = TextFromAudioDataset("rudevices_chunk/", "YourTTS-RU-RUSLAN-April-30-2023_03+48PM-0000000/config.json")

loader = DataLoader(dataset, batch_size=1)
text = next(iter(loader))

print(open(next(iter(loader.dataset.dataset)), "r").read().strip())

text_lengths = torch.LongTensor(1)
text_lengths[0] = len(text[0])
batch = text, text_lengths

print(batch)

prior_encoder(batch[0], batch[1])[0].shape

In [ ]:
decoder = TextDecoder()
_, *forward_params, _ = prior_encoder(batch[0], batch[1])
decoder(*forward_params, torch.ones((1, 10000)).long())

### Check GPT

In [ ]:
# https://github.com/RVC-Boss/GPT-SoVITS/blob/f36ca4a451e6614073d548321ad066dce73ec44d/GPT_SoVITS/configs/s2.json
hps = {"filter_length": 2048, "segment_size": 20480, "hop_length": 640, "n_speakers": 300, 
       "model": {
    "inter_channels": 192,
    "hidden_channels": 192,
    "filter_channels": 768,
    "n_heads": 2,
    "n_layers": 6,
    "kernel_size": 3,
    "p_dropout": 0.1,
    "resblock": "1",
    "resblock_kernel_sizes": [3,7,11],
    "resblock_dilation_sizes": [[1,3,5],[1,3,5],[1,3,5]],
    "upsample_rates": [10,8,2,2,2],
    "upsample_initial_channel": 512,
    "upsample_kernel_sizes": [16,16,8,2,2],
    "n_layers_q": 3,
    "use_spectral_norm": False,
    "gin_channels": 512,
    "semantic_frame_rate": "25hz",
    "freeze_quantizer": True
  }}
vq_model = SynthesizerTrn(
        hps['filter_length'] // 2 + 1,
        hps['segment_size'] // hps['hop_length'],
        n_speakers=hps['n_speakers'],
        **hps['model']
    )
# torch.load('ali/ф/rudevices_chunk.1.d.588fa2d0-7543-49d1-b8d3-99091c2edcf5.wav.content.pt')['content'].shape
hvec = torch.rand(*[1, 768, 215]) #.transpose(1,2).cpu()#torch.Size([1, 768, 215])
codes = vq_model.extract_latent(hvec)
codes.shape

In [ ]:
codes[..., :10]